# Dash Component Building

# Initialize

## Imports

In [4]:
### Setup Env for Local
import os; from importlib import reload;
%cd {os.environ['PROJECT_PATH']}
from utils.constants import *

/app


In [5]:
from dash import dcc, html, dash_table, callback
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
import plotly.express as px
import jupyter_dash

## Empty Dash App

In [25]:
px.data.iris()

,sepal_length,sepal_width,petal_length,petal_width,species,species_id
0,5.1,3.5,1.4,0.2,setosa,1
1,4.9,3.0,1.4,0.2,setosa,1
2,4.7,3.2,1.3,0.2,setosa,1
3,4.6,3.1,1.5,0.2,setosa,1
4,5.0,3.6,1.4,0.2,setosa,1
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica,3
146,6.3,2.5,5.0,1.9,virginica,3
147,6.5,3.0,5.2,2.0,virginica,3
148,6.2,3.4,5.4,2.3,virginica,3


In [55]:
df = px.data.iris()

In [56]:
df.species.unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

In [ ]:
app = jupyter_dash.JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

df = px.data.iris()

app.layout = html.Div([
    "Hello World!",
    dcc.Dropdown(id='my-input', multi=True, options=df.species.unique()),
    html.Div(id="output-container", children=build_iris_scatter()),
        # html.Div(id="output-container", children=[])

])


@app.callback(
    Output('output-container','children'),
    Input('my-input','value')
)
def build_iris_scatter(species=None):
    # return 'returning species : '+str(species)

    return [
        html.H2('You Selected : '+str(species)),
        dcc.Graph(
            id="species-scatter", 
            figure=px.scatter_3d(
                df,
                x='sepal_length',
                y='sepal_width',
                z='petal_length',
                color='species'   
            )
        )
    ]

app.run_server(mode='inline',port=8502, host="0.0.0.0", debug=True, **{'width': '800px', 'height': '400px'})  

# Components

## Building Default Component (and Conventions)

In [7]:
from dash import callback, html, dash_table, dcc, Input, Output, State, MATCH
import uuid, warnings
import pandas as pd
# from dash_app.cache import redis_store
from dash_app import cache; reload(cache); redis_store = cache.redis_store

class EmptyStoreAIO(html.Div):
    
    COMP_NAME = "EmptyStoreAIO"
    class ids:
        store_in = lambda aio_id: {'aio_id': aio_id,'component': "EmptyStoreAIO",'subcomponent': 'store_in'}
        store_out = lambda aio_id: {'aio_id': aio_id,'component': "EmptyStoreAIO",'subcomponent': 'store_out'}
        text_state = lambda aio_id: {'aio_id': aio_id,'component': "EmptyStoreAIO",'subcomponent': 'text_state'}
    
    
    def __init__(self, aio_id=None, data_in=None, store_in=None, verbose=False):
        if aio_id is None: aio_id = str(uuid.uuid4())
        if verbose : print("EmptyStoreAIO.__init__", self.COMP_NAME, aio_id, data_in, store_in, '\n')
    
        if data_in is not None: store_in = redis_store.save(data_in)
    
        super().__init__([
            dcc.Store(id=self.ids.store_in(aio_id), data=store_in),
            dcc.Store(id=self.ids.store_out(aio_id)),
            html.Pre(id=self.ids.text_state(aio_id),
                     children=["EmptyStoreAIO.rendered", self.COMP_NAME, aio_id], 
                     style={'display':'' if verbose else 'none'}),
        ])
        
    @callback([Output(ids.store_out(MATCH),'data'),Output(ids.text_state(MATCH),'children'),], 
        Input(ids.store_in(MATCH), 'data'))
    def from_in_to_out(df_key):
        print("EmptyStoreAIO.from_in_to_out", df_key, '\n')
        df_key_out = df_key
        # df = redis_store.load(df_key); 
        # df_key_out = redis_store.save(df)
        return df_key, str({'store_in' : df_key, 'store_out' : df_key_out})

    
######################
#  Use in Dash Page  #
######################

empty_component = EmptyStoreAIO('my_empty_comp_id', verbose=True, data_in=pd.DataFrame({'numbers': [1, 2, 3], 'colors': ['red', 'white', 'blue']}))

app = jupyter_dash.JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = html.Div([
    empty_component,
    html.Div(['Hello World'], id='use_component_output')
])

@app.callback(
    Output('use_component_output','children'),
    Input(empty_component.ids.store_out('my_empty_comp_id'), 'data')
)
def from_empty_to_text(df_key):
    print("from_empty_to_text", df_key, '\n')
    df = redis_store.load(df_key)
    return "DataFrame Shape : " + str(df.shape)

app.run_server(mode='external',port=8501, host="0.0.0.0", debug=True, **{'width': '800px', 'height': '400px'})  

EmptyStoreAIO.__init__ EmptyStoreAIO my_empty_comp_id    numbers colors
0        1    red
1        2  white
2        3   blue None 

Dash app running on http://0.0.0.0:8501/


/app/src/dash_app/cache.py:49: UserWarning: Using FakeRedis - Not suitable for Production Use.
  warnings.warn('Using FakeRedis - Not suitable for Production Use.')


## Building JSON TextArea (with Confirmation)

In [8]:
from dash import callback, html, dash_table, dcc, Input, Output, State, MATCH
from dash.exceptions import PreventUpdate
import uuid, warnings
import pandas as pd
# from dash_app.cache import redis_store
from dash_app import cache; reload(cache); redis_store = cache.redis_store

class JsonInputAIO(html.Div):
    
    COMP_NAME = "JsonInputAIO"
    class ids:
        store_in = lambda aio_id: {'aio_id': aio_id,'component': "JsonInputAIO",'subcomponent': 'store_in'}
        store_out = lambda aio_id: {'aio_id': aio_id,'component': "JsonInputAIO",'subcomponent': 'store_out'}
        text_state = lambda aio_id: {'aio_id': aio_id,'component': "JsonInputAIO",'subcomponent': 'text_state'}
        input_dropdown = lambda aio_id: {'aio_id': aio_id,'component': "JsonInputAIO",'subcomponent': 'input_dropdown'}
        input_textarea = lambda aio_id: {'aio_id': aio_id,'component': "JsonInputAIO",'subcomponent': 'input_textarea'}
        confirm_button = lambda aio_id: {'aio_id': aio_id,'component': "JsonInputAIO",'subcomponent': 'confirm_button'}
        
    
    
    def __init__(self, aio_id=None, options={}, store_in=None, verbose=False):
        if aio_id is None: aio_id = str(uuid.uuid4())
        if verbose : print("JsonInputAIO.__init__", self.COMP_NAME, aio_id, store_in)
    
        options_inv = {v: k for k, v in options.items()}
    
        super().__init__([
            dbc.CardBody([
                dbc.Row([dcc.Dropdown(id=self.ids.input_dropdown(aio_id), options=options_inv)]),
                dbc.Row([
                    dcc.Textarea(id=self.ids.input_textarea(aio_id), placeholder="Select Sample JSON..")
                ]),
                dbc.Row([dbc.Button(["Confirm JSON"],id=self.ids.confirm_button(aio_id))]),
            ]),
            dcc.Store(id=self.ids.store_in(aio_id), data=store_in),
            dcc.Store(id=self.ids.store_out(aio_id)),
            html.Pre(id=self.ids.text_state(aio_id),
                     children=["JsonInputAIO.rendered",' ', self.COMP_NAME,' ', aio_id], 
                     style={'display':'' if verbose else 'none'}),
        ])

    @callback(Output(ids.input_textarea(MATCH),'value'), 
        Input(ids.input_dropdown(MATCH), 'value'))
    def from_dropdown_to_textarea(value):
        print("JsonInputAIO.from_dropdown_to_textarea", value)
        # df_key_out = df_key
        # df = redis_store.load(df_key); 
        # df_key_out = redis_store.save(df)
        return value or ""

        
    @callback(Output(ids.store_out(MATCH),'data'), 
        Input(ids.confirm_button(MATCH), 'n_clicks'),
        State(ids.input_textarea(MATCH), 'value'),)
    def from_confirm_to_out(n_clicks, text_input=''):
        print("JsonInputAIO.from_confirm_to_out", text_input, n_clicks)
        if n_clicks is None:raise PreventUpdate
        return text_input

    
######################
#  Use in Dash Page  #
######################

options = {
    "Split Randomly" : '''{"asdf":5}''',
    "Split Randomly1" : '''{"asdf":51}''',
    "Split Randomly2" : '''{"asdf":52}''',
    "Split Randomly3" : '''{"asdf":53}''',
}

my_json_comp = JsonInputAIO('my_json_comp_id', options)

app = jupyter_dash.JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = html.Div([
    dbc.Card([my_json_comp], className='col-6'),
    html.Div(['Hello World!'], id='use_component_output')
])

@app.callback(
    Output('use_component_output','children'),
    Input(my_json_comp.ids.store_out('my_json_comp_id'), 'data')
)
def from_empty_to_text(df_key):
    print("page from_empty_to_text", df_key)
    # df = redis_store.load(df_key)
    return "Component Output : " + str(df_key)

app.run_server(mode='external',port=8501, host="0.0.0.0", debug=True, **{'width': '800px', 'height': '400px'})  

Dash app running on http://0.0.0.0:8501/


/app/src/dash_app/cache.py:49: UserWarning: Using FakeRedis - Not suitable for Production Use.
  warnings.warn('Using FakeRedis - Not suitable for Production Use.')
/usr/local/lib/python3.8/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning: The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.
  func()


## Building DataTable (Rough)

In [9]:


### App Setup
from dash_app.cache import redis_store
from dash_app.components import datatable_comp; reload(datatable_comp)
DataTableAIO = datatable_comp.DataTableAIO
from dash_app.components import load_data_nipams_comp; reload(load_data_nipams_comp)
PythonDataLoaderAIO = load_data_nipams_comp.PythonDataLoaderAIO
from dash_app.components import overview_data_nipams_comp; reload(overview_data_nipams_comp)
NipamsDataOverviewAIO = overview_data_nipams_comp.NipamsDataOverviewAIO


app = jupyter_dash.JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
input_data_comp = PythonDataLoaderAIO(aio_id='input_data_comp')
# filter_data_comp = DataTableAIO(df=pd.DataFrame(), aio_id='filter_data_comp')
overview_data_comp = NipamsDataOverviewAIO(aio_id='overview_data_comp')

app.layout = dbc.Col([
    input_data_comp, 
    html.Div(id='filter_table'),
    # filter_data_comp,
    overview_data_comp,
])




@app.callback(
    # Output(filter_data_comp.ids.store('filter_data_comp'),'data'),
    Output('filter_table','children'),
    Input(input_data_comp.ids.store('input_data_comp'), 'data')
)
def from_input_to_filter(df_key):
    print('render_overview_container', df_key);
    # df = redis_store.load(df_key)
    return DataTableAIO(df=df, aio_id='filter_data_comp')
    # return {"df":df_key}
    
# @app.callback(
#     Output(filter_data_comp.ids.store('overview_data_comp'),'data'),
#     # Output('filter_table','children'),
#     Input(input_data_comp.ids.store('input_data_comp'), 'data')
# )
# def from_input_to_filter2(df_key):
#     print('render_overview_container', df_key);
#     df = redis_store.load(df_key)
#     return {"df":df_key}
#     # return DataTableAIO(df=df, aio_id='filter_data_comp')

# @app.callback(
#     Output(overview_data_comp.ids.store('overview_data_comp'),'data'),
#     Input(input_data_comp.ids.store('input_data_comp'), 'data')
# )
# def from_input_to_overview(df):
#     print('render_overview_container', df);
#     return df
    
@app.callback(
    Output(overview_data_comp.ids.store('overview_data_comp'),'data'),
    Input(filter_data_comp.ids.store_out('filter_data_comp'), 'data')
)
def from_filter_to_overview(df):
    print('render_overview_container', df);
    return df
    
    
app.run_server(mode='external',port=8502, host="0.0.0.0", debug=True, **{'width': '800px', 'height': '400px'})  



nipams -  PythonDataLoaderAIO.__init__ input_data_comp


NameError: name 'filter_data_comp' is not defined

## Building Pages

### Template

### Nipams Data Overview

In [12]:
app = jupyter_dash.JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])


print('data_overview.py')
import dash
from dash import callback, html, dcc, Input, Output, State, MATCH
import dash_bootstrap_components as dbc

from utils.constants import *
from dash_app.cache import redis_store

# from app import app
# import data_table

# from dash_app.components.datatable_comp import DataTableAIO
# from dash_app.components.load_data_nipams_comp import PythonDataLoaderAIO
# from dash_app.components.overview_data_nipams_comp import NipamsDataOverviewAIO
from dash_app.components import datatable_comp; reload(datatable_comp)
DataTableAIO = datatable_comp.DataTableAIO
from dash_app.components import load_data_nipams_comp; reload(load_data_nipams_comp)
PythonDataLoaderAIO = load_data_nipams_comp.PythonDataLoaderAIO
from dash_app.components import overview_data_nipams_comp; reload(overview_data_nipams_comp)
NipamsDataOverviewAIO = overview_data_nipams_comp.NipamsDataOverviewAIO


input_data_comp = PythonDataLoaderAIO(aio_id='input_data_comp')
filter_data_comp = DataTableAIO(df=pd.DataFrame(), aio_id='filter_data_comp')
overview_data_comp = NipamsDataOverviewAIO(aio_id='overview_data_comp')


layout = dbc.Col([
    html.H1('Data Loading', style={"textAlign": "center"}),
    input_data_comp, 
    dbc.CardBody([
        html.H4("Filter DataFrame", className="card-title"),html.P("Apply Columns Filters to Update Selected Data",className="card-text"),
        dbc.Spinner([
            html.Div(id='filter_table')
        ], color='primary', spinner_style={'width':'5rem','height':'5rem'}),
    ]),
    # filter_data_comp,
    overview_data_comp,
    dbc.CardBody([
        html.H4("Finalize Selection", className="card-title", style={"textAlign": "center"}),
        dbc.Button(id='', children=['Confirm Selection'])
        # dbc.Spinner([
        #     html.Div(id='filter_table')
        # ], color='primary', spinner_style={'width':'5rem','height':'5rem'}),
    ]),
    dcc.Store(id='page_data_load_output')
])

@app.callback(
    # Output(filter_data_comp.ids.store('filter_data_comp'),'data'),
    Output('filter_table','children'),
    Input(input_data_comp.ids.store('input_data_comp'), 'data')
)
def from_input_to_filter(df_key):
    print('from_input_to_filter', df_key)
    df = redis_store.load(df_key)
    return DataTableAIO(df=df, aio_id='filter_data_comp')
    # return {"df":df_key}

@app.callback(
    Output(overview_data_comp.ids.store('overview_data_comp'),'data'),
    Input(filter_data_comp.ids.store_out('filter_data_comp'), 'data')
)
def from_filter_to_overview(df=''):
    print('from_filter_to_overview', df)
    return df

@app.callback(
    Output('page_data_load_output','data'),
    Input(filter_data_comp.ids.store_out('filter_data_comp'), 'data')
)
def from_overview_to_output(df):
    print('from_overview_to_output', df)
    return df

app.layout = layout
app.run_server(mode='inline',port=8501, host="0.0.0.0", debug=True, **{'width': '800px', 'height': '400px'})  


data_overview.py

nipams -  PythonDataLoaderAIO.__init__ input_data_comp



nipams -  input-file-pattern, button-load ['HLV', 'LLV'] 0

nipams -  input-file-pattern, button-load ['HLV', 'LLV'] 0

nipams -  input-file-pattern, button-load 6d4c810f0891bc49ddb12192c67f2f805ea84d05b85b6e764eebd8dbcfc48bb1724489574d53558ffba8985bedfe421780c5d8c5fe5520f5298a721b7a1346d3 (195376, 19)

nipams -  print_success_msg 6d4c810f0891bc49ddb12192c67f2f805ea84d05b85b6e764eebd8dbcfc48bb1724489574d53558ffba8985bedfe421780c5d8c5fe5520f5298a721b7a1346d3
from_input_to_filter 6d4c810f0891bc49ddb12192c67f2f805ea84d05b85b6e764eebd8dbcfc48bb1724489574d53558ffba8985bedfe421780c5d8c5fe5520f5298a721b7a1346d3

nipams -  input-file-pattern, button-load 6d4c810f0891bc49ddb12192c67f2f805ea84d05b85b6e764eebd8dbcfc48bb1724489574d53558ffba8985bedfe421780c5d8c5fe5520f5298a721b7a1346d3 (195376, 19)
from_input_to_filter 6d4c810f0891bc49ddb12192c67f2f805ea84d05b85b6e764eebd8dbcfc48bb1724489574d53558ffba8985bedfe421780c5d8c5fe5520f5298a721b7a1346d3

nipams -  print_success_msg 6d4c810f0891bc49ddb1219

/usr/local/lib/python3.8/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/usr/local/lib/python3.8/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/usr/local/lib/python3.8/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/usr/local/lib/python3.8/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/usr/local/lib/python3.8/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pand


nipams -  input-file-pattern, button-load ['HLV', 'LLV'] 0

nipams -  input-file-pattern, button-load 6d4c810f0891bc49ddb12192c67f2f805ea84d05b85b6e764eebd8dbcfc48bb1724489574d53558ffba8985bedfe421780c5d8c5fe5520f5298a721b7a1346d3 (195376, 19)

nipams -  print_success_msg from_input_to_filter 6d4c810f0891bc49ddb12192c67f2f805ea84d05b85b6e764eebd8dbcfc48bb1724489574d53558ffba8985bedfe421780c5d8c5fe5520f5298a721b7a1346d3
6d4c810f0891bc49ddb12192c67f2f805ea84d05b85b6e764eebd8dbcfc48bb1724489574d53558ffba8985bedfe421780c5d8c5fe5520f5298a721b7a1346d3

nipams -  datatable_comp filter  {'df': '6d4c810f0891bc49ddb12192c67f2f805ea84d05b85b6e764eebd8dbcfc48bb1724489574d53558ffba8985bedfe421780c5d8c5fe5520f5298a721b7a1346d3'}
from_filter_to_overview 6d4c810f0891bc49ddb12192c67f2f805ea84d05b85b6e764eebd8dbcfc48bb1724489574d53558ffba8985bedfe421780c5d8c5fe5520f5298a721b7a1346d3
from_overview_to_output 6d4c810f0891bc49ddb12192c67f2f805ea84d05b85b6e764eebd8dbcfc48bb1724489574d53558ffba8985bedfe42178

/usr/local/lib/python3.8/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/usr/local/lib/python3.8/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/usr/local/lib/python3.8/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/app/src/dash_app/components/overview_data_nipams_comp.py:69: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



## Nipams Splitting Page

In [ ]:
# from dash_app import cache; reload(cache); redis_store = cache.redis_store
from dash_app.components import basic_comps; reload(basic_comps)
JsonInputAIO = basic_comps.JsonInputAIO

app = jupyter_dash.JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

options_splitting = {
    'Split By Random' : '{}'
}

json_input_split = JsonInputAIO('json_input_split', options_splitting)

app.layout = dbc.Col([
    html.H1('Data Splitting', style={"textAlign": "center"}), 
    dbc.Row([
        dbc.Card([dbc.CardBody([
                html.H4("Set up Experiment", className="card-title"),
                html.Div([json_input_split]),
            ]),
        ], className='col-4'),
        dbc.Card([dbc.CardBody([
            html.Div(id='experiment-overview',children=['Experiment Output Pending..'])
        ])], className='col-8'),
    ]),
    dbc.Row([
        dbc.CardBody([
            html.H4("Finalize", className="card-title", style={"textAlign": "center"}),
            dbc.Button(id='confirm_splitting', children=['Confirm Experiment Design'])
        ], style={'textAlign':'center'}),
    ]),
    dcc.Store(id='page_data_splitting_output')
])

@app.callback(
    Output('experiment-overview','children'),
    Input(json_input_split.ids.store_out('json_input_split'), 'data')
)
def from_empty_to_text(df_key):
    print("page from_empty_to_text", df_key)
    # df = redis_store.load(df_key)
    return "Experiment Output : " + str(df_key)

@app.callback(Output('page_data_splitting_output','data'),
              Input('confirm_splitting', 'n_clicks'), State(json_input_split.ids.store_out('json_input_split'), 'data'))
def from_confirm_to_pageout(n_clicks, df_key):
    if n_clicks is None: raise PreventUpdate
    print("page from_confirm_to_pageout", df_key)
    # df = redis_store.load(df_key)
    return "Page Output : " + str(df_key)


app.run_server(mode='external',port=8501, host="0.0.0.0", debug=True, **{'width': '800px', 'height': '400px'})  

# Playground

## Fetching from Cache (Redis)

In [ ]:
import json
from dash_app.cache import redis_store
# hash_key = '6d4c810f0891bc49ddb12192c67f2f805ea84d05b85b6e764eebd8dbcfc48bb1724489574d53558ffba8985bedfe421780c5d8c5fe5520f5298a721b7a1346d3'
hash_key = '500c9dda4dacec74c219821247da63b281c609fe225d82edb2322008fdac86a1cb3c8de54c7a3a4b94d0cb2242120c515afcdb60a48a1f6b68a9fa684692fbd7'
df = redis_store.load(hash_key)
df

In [ ]:
print(df.head().to_string().replace('\n','<br/>'))

## Building Graphs

In [ ]:
df_ = df[df.file.sample().values[0] == df.file]
df_ = df_.reset_index(drop=True)

# px.line(df_.select_dtypes(include=np.number))
px.line(df_.select_dtypes(include=np.number).drop('ts',axis=1))

JsonInputAIO.from_dropdown_to_textarea None
JsonInputAIO.from_confirm_to_out None None
